In [18]:
import requests
import json
import pandas as pd
import datetime
from datetime import datetime, timezone
import hashlib
from sqlalchemy import create_engine
import numpy as np
from time import sleep
import logging
import pytz
import math
import os

In [26]:
from dotenv import load_dotenv
env_file = Path("../.env")
print(".env exists:", env_file.exists(), "→", env_file)
# Загружаем .env
loaded = load_dotenv(dotenv_path=env_file)

.env exists: True → ..\.env


In [3]:
pd.set_option('display.max_columns', None)  # Отображать все столбцы
pd.set_option('display.width', 1000)  # Увеличить ширину отображения

In [4]:
# Текущее время в московском часовом поясе
moscow_tz = pytz.timezone('Europe/Moscow')
current_time = datetime.now(moscow_tz).strftime('%Y-%m-%dT%H:%M:%S.%f%z')
# Уубираем двоеточие
current_time = current_time[:-2] + ':' + current_time[-2:]

In [32]:
url = os.getenv('URL')
headers = {
    'Content-Type': os.getenv('CONTENT_TYPE'),
    'Authorization': os.getenv('AUTHORIZATION'),
}

In [6]:
# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

Накладные в статусе "в пути

In [38]:
payload = json.dumps({
  "com": "execOperation",
  "op": "static/getList()",
  "otype": "Invoice",
  "opargs": {
    "filters": [
          {
        "column": "lcState",
        "condition": "=",
        "value": [
         'onWay'
        ]
      },
      {
         "column": "recieverContractorId",
         "condition": "in",
         "value": [248824]
            }
    ],
    "size": 200,
    "getFullCards": 0
  }
}
)


response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)

invoice_data = response.text

#print(invoice_data)

data = json.loads(invoice_data)

# Извлечем таблицу из данных
attr_table = data["resData"]["attrTable"]

# Преобразуем в DataFrame
df = pd.DataFrame(attr_table[1:], columns=attr_table[0])

display(df)

200


,id,docNum,docDate,name,lcState,contractNum,ownerContractorId,doNotChangeProprietor,shipperContractorId,sourceWarehouseId,dateReceive,receiverContractorId,destinationWarehouseId,dateReturn,dateExport,carrierContractorId,vehicleNumber,docNote,docNoteReceiver,sysGuid,sysTimeFrom,sysChangedAt,ownerContractorName,sourceWarehouseName,receiverContractorName,destinationWarehouseName
0,4817477,003,2025-04-14T21:00:00.001Z,003 от 14.04.2025,onWay,ТК_003,248824,false,,1077174,2025-04-16T10:47Z,248824,0,,,,,test_upd_number_3/2025,,,1744634636818,1764326832633,"ООО ""ЛЕ МОНЛИД""","(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)","ООО ""ЛЕ МОНЛИД""",
1,4817483,005,2025-04-14T13:19:45.019Z,005 от 14.04.2025,onWay,ТК_005,248824,false,,1077174,,248824,0,,,25365,В391СО199,test_upd_number_5/2025,,,1744636856509,1744637606595,"ООО ""ЛЕ МОНЛИД""","(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)","ООО ""ЛЕ МОНЛИД""",
2,4842430,005,2025-11-27T13:19:45.019Z,005 от 27.11.2025,onWay,ТК_005,248824,,,1077174,,248824,0,,,,В391СО199,test_upd_number_5/2025,,,1764263438998,1764263466977,"ООО ""ЛЕ МОНЛИД""","(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)","ООО ""ЛЕ МОНЛИД""",
3,4842560,Накладная_2,2025-11-28T06:20:21.328Z,Накладная_2 от 28.11.2025,onWay,,248824,,,1077174,,248824,0,,,887,В391СО197,,,,1764310868454,1764310871075,"ООО ""ЛЕ МОНЛИД""","(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)","ООО ""ЛЕ МОНЛИД""",
4,4842750,20251203-002,2025-12-03T17:54:52.489810Z,20251203-002 от 03.12.2025,onWay,,248824,,,1077174,,248824,0,,,,,Invoice_to_retail_2_items,,,1764773692609,1764833871802,"ООО ""ЛЕ МОНЛИД""","(RU-077/RD83952) 86_ТЦ ""Леруа Мерлен"" (Троицк)","ООО ""ЛЕ МОНЛИД""",


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        5 non-null      object
 1   docNum                    5 non-null      object
 2   docDate                   5 non-null      object
 3   name                      5 non-null      object
 4   lcState                   5 non-null      object
 5   contractNum               5 non-null      object
 6   ownerContractorId         5 non-null      object
 7   doNotChangeProprietor     5 non-null      object
 8   shipperContractorId       5 non-null      object
 9   sourceWarehouseId         5 non-null      object
 10  dateReceive               5 non-null      object
 11  receiverContractorId      5 non-null      object
 12  destinationWarehouseId    5 non-null      object
 13  dateReturn                5 non-null      object
 14  dateExport                5 no

Цикл для последовательного принятия накладных

In [34]:
# Счетчики для статистики
success_count = 0
error_count = 0
skipped_count = 0

In [ ]:
for index, row in df.iterrows():
    invoice_id = row['id']
    destination_warehouse_id = row['destinationWarehouseId']
    
    # Пропускаем записи с destinationWarehouseId = 0
    if destination_warehouse_id == 0:
        logger.warning(f"Пропущена запись {invoice_id}: destinationWarehouseId = 0")
        skipped_count += 1
        continue
    
    # Проверяем что значения не пустые
    if pd.isna(invoice_id) or pd.isna(destination_warehouse_id):
        logger.warning(f"Пропущена запись с индексом {index}: пустые значения")
        skipped_count += 1
        continue
    
    try:
        # Формируем payload
        payload = json.dumps({
            "com": "execOperation",
            "op": "onWay/notifyDelivered()",
            "oid": str(invoice_id),
            "otype": "Invoice", 
            "opargs": {
                'theCard': {
                    'dateAction': current_time,
                    'description': 'test_2',
                    'destinationWarehouseId': int(destination_warehouse_id)
                }
            }
        })
        
        # Запрос
        response = requests.request("POST", url, headers=headers, data=payload, timeout=30)
        
        # Логируем 
        logger.info(f"[{index + 1}/{len(df)}] ID: {invoice_id}, Склад: {destination_warehouse_id}, Status: {response.status_code}")
        
        if response.status_code == 200:
            success_count += 1
            try:
                result = response.json()
                logger.info(f"Принято: {result.get('resMsg', 'OK')}")
            except Exception as e:
                logger.error(f"Ошибка парсинга JSON: {e}")
        else:
            error_count += 1
            logger.error(f"HTTP ошибка: {response.text}")
            
    except requests.exceptions.RequestException as e:
        error_count += 1
        logger.error(f"Ошибка запроса для ID {invoice_id}: {e}")
    except Exception as e:
        error_count += 1
        logger.error(f"Неожиданная ошибка для ID {invoice_id}: {e}")
    
    # Пауза между запросами
    sleep(0.5)

In [ ]:
# Итоговая статистика
logger.info(f"Обработка завершена. Успешно: {success_count}, Ошибок: {error_count}, Пропущено: {skipped_count}")